## Import all needed packages

In [14]:
import xarray as xr
from pyproj import CRS
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, box, LineString
import folium
import os
from datetime import datetime
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
import re
import psutil
import sys
import regionmask
import cf_xarray
from pyproj import CRS
import dask.array as da


## Managing coordinates for window extraction

### Example of station information

In [15]:
# Example of stations information
# stationID = 'S1'
# station = 'SCHAARVODDL	Schaar van Ouden Doel'
# lon = 4.250932603
# lat = 51.35118367
# x_coord =  5.8711167e+05 #5.208e+05  
# y_coord =  5.68962179e+06

# stationID = 'S2'
# station = 'VLISSGNLDK	Vlissingen Nolledijk'
# lon = 3.553066,	
# lat = 51.450453
# x_coord =  5.3843131e+05   
# y_coord =  5.70006397e+06 

#### Define functions

In [16]:
# Create a geodataframe from a dataframe (.csv)
def create_gdf_points(df):
    # Create geometry component for geodataframe from a dataframe (.csv)
    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
    gdf = gdf.to_crs("EPSG:4326")

    return gdf

def create_dataframe_for_window_extraction(gdf,epsg_code):
    
    gdf = gdf.to_crs(epsg_code)
    gdf['x_coord'] = gdf['geometry'].x
    gdf['y_coord'] = gdf['geometry'].y
    gdf.insert(3, 'x_coord', gdf.pop('x_coord'))
    gdf.insert(4, 'y_coord', gdf.pop('y_coord'))
    gdf.insert(0, 'stationID', gdf.index)
    gdf.insert(2, 'lon', gdf['longitude'])
    gdf.insert(3, 'lat', gdf['latitude'])
    gdf = gdf.rename(columns={'stationname': 'station'})
    columns_to_remove = ['longitude', 'latitude', 'geometry']
    df_for_window_extraction = gdf.drop(columns=columns_to_remove)

    return df_for_window_extraction

def create_map(gdf):
    # Get the center of the map based on the points
    center_lat, center_lon = gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()

    # Create a Folium map centered at the mean coordinates
    mymap = folium.Map(location=[center_lat, center_lon], zoom_start=10)

    return mymap


def add_points_to_map(gdf, mymap):

    # Add GeoPoints to the map
    # for idx, row in gdf.iterrows():
    #    folium.Marker(
    #        [row.geometry.y, row.geometry.x], 
    #        popup=f"Point {idx}"
    #    ).add_to(mymap)

    # Add Circle markers to the map
    for idx, row in gdf.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,  # Adjust the radius as needed
            color='blue',  # Circle color
            fill=True,
            fill_color='blue',  # Fill color
            fill_opacity=0.6,
            popup=f"Point {idx}"
        ).add_to(mymap)

    return mymap

def add_boundingbox_to_map(mymap):
    # Define limits in Westerschelde
    # limit=51.2,3.3,51.55,4.4

    # Create a bounding box from the given limits
    min_lat, min_lon, max_lat, max_lon = 51.2, 3.1, 51.55, 4.4
    bbox = box(min_lon, min_lat, max_lon, max_lat)

    # Create a GeoDataFrame with the bounding box and set the CRS
    gdf_bbox = gpd.GeoDataFrame(geometry=[bbox], crs='EPSG:4326')

    # Add GeoDataFrame to the map
    folium.GeoJson(gdf_bbox).add_to(mymap)

    # Display the map
    return mymap 

def save_stations_dataframe(df, output_file):
    # Save the GeoDataFrame as .xlsx file
    df.to_excel(output_file, index=False)

def save_stations_gdf(gdf, output_file):
    # Save the GeoDataFrame as GeoJSON
    gdf.to_file(output_file, driver='GeoJSON')

#### Execute functions

In [17]:
# Define stations csv file
input_file_stations_dataframe      = r'P:\11209243-eo\Window_extraction\INPUT\stationLocations_oppWater.csv'
output_file_stations_geodataframe  = r'P:\11209243-eo\Window_extraction\OUTPUT\points_stations.geojson'
output_file_stations_dataframe     = r'P:\11209243-eo\Window_extraction\OUTPUT\Stations.xlsx'

epsg_code = 32631 # Value obtained in the Explore L2W datasets (with clorophyll) section"

# Read stations csv file
df  = pd.read_csv(input_file_stations_dataframe , sep=';')

# Create geometry and save the points as geojson
gdf = create_gdf_points(df)
df_for_window_extraction = create_dataframe_for_window_extraction(gdf, epsg_code)
save_stations_dataframe(df_for_window_extraction, output_file_stations_dataframe)
save_stations_gdf(gdf, output_file_stations_geodataframe)

# Display map. Only available in jupyter notebooks
mymap = create_map(gdf)
mymap = add_points_to_map(gdf, mymap)
mymap = add_boundingbox_to_map(mymap)
mymap

C:\Users\fuentesm\AppData\Local\Temp\ipykernel_2040\2346132786.py:28: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat, center_lon = gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()


## Managing areas for window extraction

#### Define functions

In [18]:
def create_gdf_polygons(df, x_column, y_column, group_column):
    # Create geometry component for geodataframe from a dataframe (.csv)
    geometry = [Point(xy) for xy in zip(df[x_column], df[y_column])]

    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:28992")
    gdf = gdf.to_crs("EPSG:4326")

    # Define column that indicates the groups of polygons 
    gdf['Group'] = gdf[group_column] 

    # Create a dictionary to store polygons based on group values
    polygons = {}

    # Iterate over unique group values and create polygons
    for group_value in range(1, 10):
        group_points = gdf[gdf['Group'] == group_value]
        
        # Ensure there are at least three points to form a polygon
        if len(group_points) >= 3:
            # Create a Polygon from the points
            polygon = Polygon(group_points['geometry'])
            polygons[group_value] = polygon

    # Create a new GeoDataFrame with the created Polygons
    gdf_polygons = gpd.GeoDataFrame({'Group': list(polygons.keys()), 'geometry': list(polygons.values())},
                                    geometry='geometry', crs="EPSG:4326")

    return gdf_polygons

def add_polygons_to_map(mymap,polygons_gdf):

    # Add Polygons to the map
    for idx, row in polygons_gdf.iterrows():
        folium.GeoJson(row['geometry'].__geo_interface__, popup=f"Group {row['Group']}").add_to(mymap)

    return mymap

def save_polygons_gdf(polygons_gdf, output_file):
    # Save the GeoDataFrame as GeoJSON
    polygons_gdf.to_file(output_file, driver='GeoJSON')


#### Execute functions

In [19]:
# Define polygons csv file and output geojson file
input_file_polygons_dataframe      = r'P:\11209243-eo\Window_extraction\INPUT\vakkenBewerkt.csv'
output_file_polygons_geodataframe  = r'P:\11209243-eo\Window_extraction\OUTPUT\polygons_NEOZ.geojson'

# Read polygons csv file
df = pd.read_csv(input_file_polygons_dataframe)

# Create geometries
gdf_polygons = create_gdf_polygons(df, 'POINT_X', 'POINT_Y', 'CompNr')

# Save geometries
save_polygons_gdf(gdf_polygons, output_file_polygons_geodataframe)

# Display geometries
mymap = add_polygons_to_map(mymap, gdf_polygons)
mymap